# K-Nearest Neighbour (KNN) - Classification

In [409]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

path = "DataSets\diabetes.csv"
test_path = "DataSets\diabetes.csv"

In [619]:
def preprocess(dataset):
    data = dataset.values
    dataset = dataset.convert_objects(convert_numeric=True)
    columns_idx = list(dataset.iloc[:,:-1].columns)
    numeric_cols_idx = list(dataset.iloc[:,:-1]._get_numeric_data().columns)
    categorical_cols_idx = list(set(columns_idx) - set(numeric_cols_idx))

    Y = data[1:,-1]
    
    lb = preprocessing.LabelBinarizer()
    Y = pd.DataFrame(lb.fit_transform(Y), columns=[-1])
    for i in categorical_cols_idx:
        dataset.iloc[:, i] = pd.Categorical(dataset.iloc[:, i])
        dataset.iloc[:, i] = dataset.iloc[:, i].cat.codes
    
    return dataset.iloc[:,:-1], Y

In [549]:
def load_data(path):
    dataset = pd.read_csv(path, header=None)
    data, Y = preprocess(dataset)
    
    data.reset_index(drop=True, inplace=True)
    Y.reset_index(drop=True, inplace=True)
    
    return data.iloc[1:,:], Y

In [550]:
training_data, Y = load_data(path)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  This is separate from the ipykernel package so we can avoid doing imports until


In [612]:
def KNN(training_data, Y, Observation, n_neighbours):

    distance_array, index_array = np.array([]), np.array([])
    Observation = np.float_(Observation)
    
    distance_array = pd.DataFrame(np.sqrt(np.square((training_data - Observation)).sum(axis=1)), columns=["index"])
    sorted_array = distance_array.sort_values(["index"], ascending=1)
    K_neighbours = sorted_array.index[:n_neighbours]
    Y_values = np.array(Y.iloc[K_neighbours])
    
    #print(Y, distance_array)
    vote = 0 if len(Y_values[Y_values == 0]) >= len(Y_values[Y_values == 1]) else 1

    return vote

In [613]:
#training_data.iloc[17], KNN(training_data, Y, training_data.iloc[17], 3)

In [614]:
def Accuracy(training_data, Y_train, Observations, Y):
    total, TP, FP, TN, FN, prediction, Accuracy = 0, 0, 0, 0, 0, 0, 0
    Observations = Observations.astype('str')
    n_neighbours = 3
    
    Observations.reset_index(drop=True, inplace=True)
    Y.reset_index(drop=True, inplace=True)
    training_data.reset_index(drop=True, inplace=True)
    Y_train.reset_index(drop=True, inplace=True)
        
    for i, row in Observations.iterrows():
        prediction = 1 if Y.iloc[i,-1] == KNN(training_data, Y_train, row, n_neighbours) else 0

        if prediction:
            total += 1
            TP += 1 if Y.iloc[i,-1] == 1 else 0
            TN += 1 if Y.iloc[i,-1] == 0 else 0
        else:
            FP += 1 if Y.iloc[i,-1] == 0 else 0
            FN += 1 if Y.iloc[i,-1] == 1 else 0

    Accuracy = total/ len(Y) * 100

    return ("Accuracy; %.4f, TP: %s, FP: %s, TN: %s, FN: %s " % (Accuracy, TP, FP, TN, FN))

In [615]:
def TransformDataSet(path):
    trasformed_data, Y = load_data(path)
    return trasformed_data, Y

In [616]:
Xtest, Ytest = TransformDataSet(test_path)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  This is separate from the ipykernel package so we can avoid doing imports until


In [617]:
Accuracy(training_data, Y, Xtest, Ytest)

'Accuracy; 85.9375, TP: 201, FP: 41, TN: 459, FN: 67 '